<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import urllib.parse as req

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
n = 10
query = f'''SELECT * 
            FROM public.vacancies 
            LIMIT {n}
         '''

In [4]:
conn_url = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}'
pd.read_sql(query, conn_url)

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811
1,55843014,Системный администратор,Средства криптографической защиты информации\t...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,130,4903713
2,54525964,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,160,69961
3,54525965,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,159,69961
4,55354053,Специалист службы поддержки с техническими зна...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,1955,1740
5,55906072,Корректор,Корректура текстов\tГрамотность\tРабота в кома...,Полный день,Нет опыта,Полная занятость,NaN,NaN,2323,584934
6,55523207,React Front-end Middle Developer,JavaScript\tTypeScript\tCSS3\tNode.js\tHTML5,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,2759,204511
7,55050261,Инженер-программист,Java SE\tSpring Framework\tSpring Boot\tGit\tS...,Полный день,Нет опыта,Полная занятость,40000.0,60000.0,49,4448636
8,55583432,Системный администратор,Linux\tАдминистрирование сетевого оборудования...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,NaN,3,55126
9,55312414,Ведущий специалист группы разработки систем,1С программирование\t1С: Предприятие 8\t1С: До...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,47,1413754


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''SELECT COUNT(id)
                FROM vacancies
            '''

In [6]:
query_3_1 = pd.read_sql(query_3_1, conn_url)
print(f"Количество вакансий в базе данных: {query_3_1.loc[0, 'count']}")

Количество вакансий в базе данных: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
# текст запроса
query_3_2 = f'''SELECT COUNT(id)
                FROM employers
            '''

In [8]:
# результат запроса
query_3_2 = pd.read_sql(query_3_2, conn_url)
print(f"Количество работодателей: {query_3_2.loc[0, 'count']}")

Количество работодателей: 23501


3. Посчитайте с помощью запроса количество регионов (таблица areas).

In [9]:
# текст запроса
query_3_3 = f'''SELECT COUNT(id)
                FROM areas
            '''

In [10]:
# результат запроса
query_3_3 = pd.read_sql(query_3_3, conn_url)
print(f"Количество регионов: {query_3_3.loc[0, 'count']}")

Количество регионов: 1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
# текст запроса
query_3_4 = f'''SELECT COUNT(DISTINCT(id))
                FROM industries
            '''

In [12]:
# результат запроса
query_3_4 = pd.read_sql(query_3_4, conn_url)
print(f"Количество сфер деятельности: {query_3_4.loc[0, 'count']}")

Количество сфер деятельности: 294


Итак, в наших данных представлено 49197 вакансий от 23501 различного работодателя. География присутствия также достаточно широкая - 1362 региона, включая города и населенные пункты из разных стран (не только Россия). Вакансии работодателей разнообразные и распределены по 294 сферам деятельности. В среднем, в каждой компании открыты как минимум две вакансии.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query_4_1 = f'''SELECT a.name AS area,
                COUNT(v.id) AS cnt
                FROM vacancies v
                JOIN areas a ON v.area_id = a.id
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 5
            '''

In [14]:
# результат запроса
query_4_1 = pd.read_sql(query_4_1, conn_url)
query_4_1

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f'''SELECT COUNT(id)
                FROM vacancies
                WHERE (salary_from IS NOT NULL AND salary_to IS NOT NULL)
                OR (salary_from IS NOT NULL AND salary_to IS NULL)
                OR (salary_from IS NULL and salary_to IS NOT NULL)
            '''

In [16]:
# результат запроса
query_4_2 = pd.read_sql(query_4_2, conn_url)
print(f"Количество вакансий: {query_4_2.loc[0, 'count']}")

Количество вакансий: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f'''SELECT AVG(salary_from) AS min,
                AVG(salary_to) AS max
                FROM vacancies
            '''

In [18]:
# результат запроса
query_4_3 = pd.read_sql(query_4_3, conn_url)
print(f"Среднее значение по нижней границе: {round(query_4_3.loc[0, 'min'])} руб.")
print(f"Среднее значение по верхней границе: {round(query_4_3.loc[0, 'max'])} руб.")

Среднее значение по нижней границе: 71065 руб.
Среднее значение по верхней границе: 110537 руб.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query_4_4 = f'''SELECT schedule,
                employment,
                COUNT(id) AS vacancies_cnt
                FROM vacancies
                GROUP BY 1, 2
                ORDER BY 3 DESC
            '''

In [20]:
# результат запроса
query_4_4 = pd.read_sql(query_4_4, conn_url)
query_4_4

,schedule,employment,vacancies_cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query_4_5 = f'''SELECT experience,
                COUNT(id) AS vacancies_cnt
                FROM vacancies
                GROUP BY 1
                ORDER BY 2
            '''

In [22]:
# результат запроса
query_4_5 = pd.read_sql(query_4_5, conn_url)
query_4_5

,experience,vacancies_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


**Большинство вакансий сосредоточено в Москве и крупных городах**, таких как Санкт-Петербург, Новосибирск, Минск. При этом половина работодателей не указывает зарплатную вилку при подборе сотрудников, а определяет ценность каждого кандидата в индивидуальном порядке при собеседовании, исходя из имеющегося опыта соискателя, его умений и навыков. В среднем, **работодатели готовы оплачивать труд своих специалистов в диапазоне от 71 до 110 тыс. руб**.

**Подавляющее число компаний проводит набор сотрудников на полный рабочий день и предлагает осуществлять деятельность на территории работодателя** - таких вакансий более 35 тыс. Следующим по популярности рабочим графиком является удаленная работа - некоторые компании привыкли к такому взаимодействию с сотрудниками, особенно это характерно для IT-компаний и онлайн маркетинга.

**Большинство компаний ищет сотрудников с минимальным опытом работы от 1 года до 3 лет**. Вакансий без опыта работы меньше в 4 раза - чуть более 7 тыс. На сайте HeadHunter также можно найти вакансии для сотрудников с высокой квалификацией и солидным опытом работы более 6 лет - таких вакансий 1337.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
# текст запроса
query_5_1 = f'''SELECT e.name AS employer,
                COUNT(v.id) AS vacancies_cnt
                FROM vacancies v
                JOIN employers e ON v.employer_id = e.id
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 5
            '''

In [24]:
# результат запроса
df = pd.read_sql(query_5_1, conn_url)
# создаем новый столбец с индексами для визуального удобства
df.index = range(1, len(df)+1)
df = df.iloc[[0, 4]]
df

,employer,vacancies_cnt
1,Яндекс,1933
5,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:
# текст запроса
query_5_2 = f'''SELECT a.name AS area,
                COUNT(DISTINCT e.id) AS employers_cnt,
                COUNT(DISTINCT v.id) AS vacancies_cnt
                FROM areas a
                JOIN employers e ON a.id = e.area
                LEFT JOIN vacancies v ON v.area_id = a.id
                GROUP BY a.id
                ORDER BY vacancies_cnt, employers_cnt DESC 
            '''

In [26]:
# результат запроса
query_5_2 = pd.read_sql(query_5_2, conn_url)
query_5_2

,area,employers_cnt,vacancies_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
649,Алматы,721,1892
650,Новосибирск,573,2006
651,Минск,1115,2112
652,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
# текст запроса
query_5_3 = f'''SELECT DISTINCT e.name AS employer,
                COUNT(DISTINCT a.id) AS areas_cnt
                FROM vacancies v
                JOIN employers e ON v.employer_id = e.id
                JOIN areas a ON v.area_id = a.id
                GROUP BY e.id
                ORDER BY 2 DESC
            '''

In [28]:
# результат запроса
query_5_3 = pd.read_sql(query_5_3, conn_url)
query_5_3

,employer,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14799,Яроцкий Е. В./ MoonX,1
14800,Ясли-сад №374 г. Минска,1
14801,ЯСП,1
14802,Ястро-Переработка,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
# текст запроса
query_5_4 = f'''SELECT COUNT(DISTINCT e.id)
                FROM employers e
                LEFT JOIN employers_industries ei ON e.id = ei.employer_id
                WHERE ei.industry_id IS NULL
            '''

In [30]:
# результат запроса
query_5_4 = pd.read_sql(query_5_4, conn_url)
print(f"Количество работодателей: {query_5_4.loc[0, 'count']}")

Количество работодателей: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
# текст запроса
query_5_5 = f'''SELECT e.name AS name,
                COUNT(DISTINCT ei.industry_id) AS industries_cnt
                FROM employers e
                LEFT JOIN employers_industries ei ON e.id = ei.employer_id
                GROUP BY e.id
                HAVING COUNT(DISTINCT ei.industry_id) = 4
                ORDER BY 1
            '''

In [32]:
# результат запроса
df = pd.read_sql(query_5_5, conn_url)
df.index = range(1, len(df)+1)
df = df.iloc[[2]]
df

,name,industries_cnt
3,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
# текст запроса
query_5_6 = f'''SELECT COUNT(ei.employer_id)
                FROM industries i
                LEFT JOIN employers_industries ei ON i.id = ei.industry_id
                WHERE i.name = 'Разработка программного обеспечения'
            '''

In [34]:
# результат запроса
query_5_6 = pd.read_sql(query_5_6, conn_url)
print(f"Количество работодателей: {query_5_6.loc[0, 'count']}")

Количество работодателей: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
# код для получения списка городов-милионников
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')

# преобразование кириллицы в url
url = path_common.scheme + '://' + req.quote(path_common.netloc) \
+ req.quote(path_common.path) + '#' + req.quote(path_common.fragment)

df = pd.read_html(url)[1]
cities = list(df['Город'])

# приведение структуры данных к единому формату
cities[13] = 'Воронеж'
cities = tuple(cities)
cities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [36]:
# текст запроса
query_5_7 = f'''SELECT a.name AS city,
                COUNT(v.id) AS vacancies_cnt
                FROM vacancies v
                LEFT JOIN areas a ON v.area_id = a.id
                LEFT JOIN employers e ON v.employer_id = e.id
                WHERE a.name IN {str(cities)} AND e.name = 'Яндекс'
                GROUP BY a.id
                
                UNION ALL
                
                SELECT 'Total',
                COUNT(v.id) AS vacancies_cnt
                FROM vacancies v
                LEFT JOIN areas a ON v.area_id = a.id
                LEFT JOIN employers e ON v.employer_id = e.id
                WHERE a.name IN {str(cities)} AND e.name = 'Яндекс'
                ORDER BY 2 
            '''

In [37]:
# результат запроса
query_5_7 = pd.read_sql(query_5_7, conn_url)
query_5_7

,city,vacancies_cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


**Большинство вакансий размещают крупнейшие компании, такие как Яндекс, Ростелеком, Газпром нефть**. У данных организаций большой штат сотрудников и множество подразделений, поэтому им постоянно требуются новые работники как на постоянной, так и на проектной основе. 

Яндекс имеет филиалы в других странах и практикует удаленный формат работы, поэтому география поиска людей в компанию у него наиболее широкая - 181 регион, включая страны ближнего зарубежья. 

**Основная часть работодателей все же ориентирована на российский рынок**. При этом некоторые работодатели указывают сразу несколько сфер деятельности (как, например, 2ГИС), а треть из них не указывают ее вовсе - в базе данных представлено 8419 таких организаций. 3553 работодателя в качестве сферы деятельности указали разработку программного обеспечения - это одна из самых востребованных и хорошо оплачиваемых сфер на сегодняшний день.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [38]:
# текст запроса
query_6_1 = f'''SELECT COUNT(id)
                FROM vacancies
                WHERE name ILIKE '%%data%%' OR name ILIKE '%%данн%%'
            '''

In [39]:
# результат запроса
query_6_1 = pd.read_sql(query_6_1, conn_url)
print(f"Количество вакансий: {query_6_1.loc[0, 'count']}")

Количество вакансий: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [40]:
# текст запроса
query_6_2 = f'''SELECT COUNT(id)
                FROM vacancies
                WHERE 
                ((name ILIKE '%%junior%%') OR
                (experience ILIKE '%%нет опыта%%') OR
                (employment ILIKE '%%стажировка%%'))
                AND
                ((name ILIKE '%%data scientist%%') OR 
                (name ILIKE '%%data science%%') OR
                (name ILIKE '%%исследователь данных%%') OR
                (name LIKE '%%ML%%' AND name NOT ILIKE '%%html%%') OR
                (name ILIKE '%%machine learning%%') OR
                (name ILIKE '%%машинн%%обучен%%'))
            '''

In [41]:
# результат запроса
query_6_2 = pd.read_sql(query_6_2, conn_url)
print(f"Количество вакансий: {query_6_2.loc[0, 'count']}")

Количество вакансий: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [42]:
# текст запроса
query_6_3 = f'''SELECT COUNT(id)
                FROM vacancies 
                WHERE
                ((name ILIKE '%%data scientist%%') OR 
                (name ILIKE '%%data science%%') OR
                (name ILIKE '%%исследователь данных%%') OR
                (name LIKE '%%ML%%' AND name NOT ILIKE '%%html%%') OR
                (name ILIKE '%%machine learning%%') OR
                (name ILIKE '%%машинн%%обучен%%'))
                AND 
                ((key_skills ILIKE '%%SQL%%') OR 
                (key_skills ILIKE '%%postgres%%'))
            '''

In [43]:
# результат запроса
query_6_3 = pd.read_sql(query_6_3, conn_url)
print(f"Количество вакансий: {query_6_3.loc[0, 'count']}")

Количество вакансий: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [44]:
# текст запроса
query_6_4 = f'''SELECT COUNT(id)
                FROM vacancies 
                WHERE
                ((name ILIKE '%%data scientist%%') OR 
                (name ILIKE '%%data science%%') OR
                (name ILIKE '%%исследователь данных%%') OR
                (name LIKE '%%ML%%' AND name NOT ILIKE '%%html%%') OR
                (name ILIKE '%%machine learning%%') OR
                (name ILIKE '%%машинн%%обучен%%'))
                AND 
                (key_skills ILIKE '%%Python%%') 
            '''

In [45]:
# результат запроса
query_6_4 = pd.read_sql(query_6_4, conn_url)
print(f"Количество вакансий: {query_6_4.loc[0, 'count']}")

Количество вакансий: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
query_6_5 = f'''SELECT 
                round(AVG(LENGTH(key_skills)-LENGTH(replace(key_skills, CHR(9), ''))+1), 2)
                FROM vacancies
                WHERE
                ((name ILIKE '%%data scientist%%') OR 
                (name ILIKE '%%data science%%') OR
                (name ILIKE '%%исследователь данных%%') OR
                (name LIKE '%%ML%%' AND name NOT ILIKE '%%html%%') OR
                (name ILIKE '%%machine learning%%') OR
                (name ILIKE '%%машинн%%обучен%%'))
             '''

In [47]:
query_6_5 = pd.read_sql(query_6_5, conn_url)
print(f"Average Key Skills: {query_6_5.loc[0, 'round']}")

Average Key Skills: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [48]:
# текст запроса
query_6_6 = f'''SELECT DISTINCT experience, 
                round(AVG(COALESCE((salary_from + salary_to)/2, 
                COALESCE(salary_from, salary_to)))) AS avg_salary
                FROM vacancies
                WHERE (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                AND
                ((name ILIKE '%%data scientist%%') OR 
                (name ILIKE '%%data science%%') OR
                (name ILIKE '%%исследователь данных%%') OR
                (name LIKE '%%ML%%' AND name NOT ILIKE '%%html%%') OR
                (name ILIKE '%%machine learning%%') OR
                (name ILIKE '%%машинн%%обучен%%'))
                GROUP BY 1
             '''

In [49]:
# результат запроса
query_6_6 = pd.read_sql(query_6_6, conn_url)
print(f"Средняя зарплата: {int(query_6_6.loc[2, 'avg_salary'])} руб.")

Средняя зарплата: 243115 руб.


В базе данных представлена **1771 вакансия для специалистов по работе с данными**. Что касается обязательных и распространенных требований работодателей к потенциальным сотрудникам – это **знание Python (351 вакансия) и SQL (201 вакансия)**. **51 вакансия ориентирована на новичков уровня junior**. При росте уровня специалиста соответственно растет и его заработная плата. В целом, оплата труда DS специалистов выше по рынку относительно других специальностей, не относящихся к данным, что свидетельствует об острой востребованности таких профессионалов на рынке труда. Data Scientist с опытом работы от 3 до 6 лет может в среднем рассчитывать на сумму в 243 тыс. руб.  

# Общий вывод по проекту

Современный рынок труда предоставляет много вариантов трудоустройства и реализации в карьере. **Больше всего возможностей имеется в мегаполисах и городах** – миллионниках, таких как Москва, Санкт-Петербург, Новосибирск, Минск. Огромное количество вакансий открывают такие **крупнейшие компании как Яндекс, Газпром, Ростелеком** – у них множество подразделений, которые постоянно нуждаются в новых сотрудниках, не боящихся развиваться вместе с компанией и брать на себя ответственность.

При этом **удаленный формат работы с каждым днем становится все популярнее** и отвоевывает позиции у графика с полным рабочим днем и работой на территории работодателя. На момент проведения анализа соотношение составляет чуть менее 8 тыс. позиций против 35 тыс. Работодатели ощутили на себе все экономические преимущества взаимодействия с сотрудниками в гибридном и удаленном форматах. Кроме того, рынок труда меняется с огромной скоростью, а рабочие места автоматизируются. 

Чтобы подтвердить это данными, выведем в какой сфере деятельности сейчас больше всего вакансий:

In [50]:
query_7_1 = f'''SELECT DISTINCT i.name,
                COUNT(v.id) AS vacancies_cnt
                FROM vacancies v
                LEFT JOIN employers_industries ei ON v.employer_id = ei.employer_id
                LEFT JOIN industries i ON i.id = ei.industry_id
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 1
            '''

In [51]:
query_7_1 = pd.read_sql(query_7_1, conn_url)
print(f"Вакансий в сфере '{query_7_1.loc[0, 'name']}: {query_7_1.loc[0, 'vacancies_cnt']}")

Вакансий в сфере 'Разработка программного обеспечения: 12499


В базе данных **доминируют технические вакансии** (четверть от всех представленных,  а IT – персонал является самым востребованным среди работодателей. Отдельную категорию составляют специалисты по анализу данных . Работа с данными  не привязана к конкретной индустрии и потребность в профессионалах, разбирающихся в Data Science, будет только возрастать. **Работодатели не предъявляют высокие требования к опыту работы соискателя – гораздо важнее его развитые профессиональные умения и навыки**.

**В целом, тенденция увеличения количества вакансий без обязательного и ежедневного присутствия в офисе будет не просто сохраняться, а возрастать**. Через некоторое время можно провести повторное исследование и зафиксировать, насколько изменилось соотношение вакансий для «удаленщиков» по отношению к вакансиям традиционного формата.